In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("tsd_train.csv")

In [3]:
train.head()

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."


In [4]:
spans = []

for i, row in train.iterrows():
    to_append_list = [int(x) for x in str(train["spans"].iloc[i])[1:len(train["spans"].iloc[i])-1].split(", ") if len(x) > 0]
    spans.append(to_append_list)

In [5]:
import more_itertools as mit
span_grouped = []

for i in range(len(spans)):
    span_grouped.append([list(group) for group in mit.consecutive_groups(spans[i])])

In [6]:
final = []

for i in range(len(span_grouped)):
    span_grouped_just_group_sentence = []
    for ranges in span_grouped[i]:
        span_grouped_just_group_groups = []
        if len(ranges) > 0:
            span_grouped_just_group_groups = [ranges[0],ranges[-1]]
        
        span_grouped_just_group_sentence.append(span_grouped_just_group_groups)
        
    final.append(span_grouped_just_group_sentence)     

In [7]:
texts = list(train["text"])

In [8]:
final_answer = []

for text,spans in zip(texts,final):
    span_word = []
    sentence = text.split(" ")
    for span in spans:
        span_word_span = []
        length_covered = -1
        i=0
        while length_covered < span[0]:
            length_covered += len(sentence[i]) +1
            i+=1
        span_word_span.append(i-1)
        
        while length_covered < span[1]:
            length_covered += len(sentence[i]) +1
            i+=1
        span_word_span.append(i-1)
        
        span_word.append(span_word_span)
    
    final_answer.append(span_word)

In [10]:
import re 
  
# Function checks if the string 
# contains any special character 
def run(string):
    regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')    
    if(regex.search(string) == None): 
        return False 
          
    else: 
        return True



def get_me_index(original,to_find,range_to_find,count_spaces,dict_of_texts,dict_of_texts_2,flag):
    #print("lolwa")
    #if flag:
    #    print(original)
    length = len(original)
    if range_to_find[0] == 0:
        original_broken = " ".join(original)[range_to_find[0]:].split(" ")
    elif range_to_find[0] > len(to_find):
        if (range_to_find[0] - len(to_find) - count_spaces) >= 0:
            original_broken = " ".join(original)[range_to_find[0] - len(to_find) - count_spaces:].split(" ")
        else:
            original_broken = " ".join(original)[0:].split(" ")
    else:
        original_broken = " ".join(original)[range_to_find[0] - 1:].split(" ")

    for i in range(len(original_broken)):
        if to_find in original_broken[i]:
            index_final = i
            #new change
            break
            
    leave_it = len(original) - len(original_broken)
    count_words = 0

    for i in range(leave_it,len(original)):
        if (to_find in original[i]) and (to_find in original_broken[index_final]) :
            count_words += 1
            return i, dict_of_texts, dict_of_texts_2
            
def get_discovery(span,group,count_spaces):
    
    combined_sentence = " ".join(span)
    
    if group[0] > 0:
        if (group[0]-1-count_spaces) >= 0:
            waste_part = combined_sentence[0:group[0]-1-count_spaces]
        else:
            waste_part = combined_sentence[0:group[0]-1]
    else:
        waste_part = combined_sentence[0:0]

                
    return len(waste_part.split(" "))

In [ ]:
import re
sentences = []
targets = []

for i in range(len(texts)):
    
    flag = False
    
    if i==199:
        flag = True
        
    x = texts[i]
    
    x_modified = re.sub(' +', ' '," ".join(text.replace("\n"," ") for text in x.split(" ")))
    
    split_x = x_modified.split(" ")
        
    count_spaces = len(x) - len(x_modified)

    
    y = ["O" for i in range(len(split_x))]
    grouped_y = final[i]
    
    dict_of_texts = {}
    
    dict_of_texts_2 = {}
    
    for group in grouped_y:
        
        span = x[group[0]:group[1]+1]
        
        split_span = span.split(" ")
                
        split_span = re.sub(' +', ' '," ".join(text.replace("\n"," ") for text in split_span)).split(" ")
        
        for item in split_span:
            if item in dict_of_texts:
                dict_of_texts[item] += 1
            else:
                dict_of_texts[item] = 1
                
            dict_of_texts_2[item] = 0
            
    
    for group in grouped_y:

        span = x[group[0]:group[1]+1]
        
        split_span = span.split(" ")
        
        split_span = re.sub(' +', ' '," ".join(text.replace("\n"," ") for text in split_span)).split(" ")
            
        discovered = 0
 
        if len(split_span) > 1:
            
            if len(split_span) > 2:
                
                for j in range(len(split_span)):
                    if j == 0:
                        if split_span[j] in split_x[get_discovery(split_x,group,count_spaces) + discovered -1:]:
                            
                            y[split_x.index(split_span[j],get_discovery(split_x,group,count_spaces) + discovered -1)] = "B"

                            #new
                            dict_of_texts[split_span[j]] -= 1
                            dict_of_texts_2[split_span[j]] += 1
                            
                        else:
                            index,dict_of_texts,dict_of_texts_2 = get_me_index(split_x,split_span[j],group,count_spaces,dict_of_texts,dict_of_texts_2,flag)
                            y[index] = "B"
                            
                    elif j == (len(split_span) - 1):
                        if flag:
                            print(group)
                        if split_span[j] in split_x[get_discovery(split_x,group,count_spaces) + discovered -1:]: 
                            y[split_x.index(split_span[j],get_discovery(split_x,group,count_spaces) + discovered -1)] = "L"
                            #new
                            dict_of_texts[split_span[j]] -= 1
                            dict_of_texts_2[split_span[j]] += 1
                        else:
                            index,dict_of_texts,dict_of_texts_2 = get_me_index(split_x,split_span[j],group,count_spaces,dict_of_texts,dict_of_texts_2,flag)
                            y[index] = "L"
                    else:
                        if split_span[j] in split_x[get_discovery(split_x,group,count_spaces) + discovered -1:]:
                            y[split_x.index(split_span[j],get_discovery(split_x,group,count_spaces) + discovered -1)] = "I"
                            #new
                            dict_of_texts[split_span[j]] -= 1
                            dict_of_texts_2[split_span[j]] += 1
                            
                        else:
                            index,dict_of_texts,dict_of_texts_2 = get_me_index(split_x,split_span[j],group,count_spaces,dict_of_texts,dict_of_texts_2,flag)
                            y[index] = "I"
                        
                    discovered += 1
                        
            elif len(split_span) == 2:
                
                
                if split_span[0] in split_x[get_discovery(split_x,group,count_spaces) + discovered -1:]:
                    y[split_x.index(split_span[0],get_discovery(split_x,group,count_spaces) + discovered -1)] = "B"
                    #new
                    dict_of_texts[split_span[0]] -= 1
                    dict_of_texts_2[split_span[0]] += 1
                else:
                    index,dict_of_texts,dict_of_texts_2 = get_me_index(split_x,split_span[0],group,count_spaces,dict_of_texts,dict_of_texts_2,flag)
                    y[index] = "B"
                #new change   
                discovered += 1

                if split_span[1] in split_x[get_discovery(split_x,group,count_spaces) + discovered -1:]:
                    if flag:
                        print("Hola")
                        print(split_x[get_discovery(split_x,group,count_spaces) + discovered -1:])
                    y[split_x.index(split_span[1],get_discovery(split_x,group,count_spaces) + discovered -1)] = "L"
                    #new
                    dict_of_texts[split_span[1]] -= 1
                    dict_of_texts_2[split_span[1]] += 1
                else:
                    if flag:
                        print("lol")
                    index,dict_of_texts,dict_of_texts_2 = get_me_index(split_x,split_span[1],group,count_spaces,dict_of_texts,dict_of_texts_2,flag)
                    y[index] = "L"
                #new change  
                discovered += 1
                
                
        elif len(split_span) == 1:
            
            if split_span[0] in split_x[get_discovery(split_x,group,count_spaces) + discovered -1:]:
                #if flag:
                #    print("lol")
                y[split_x.index(split_span[0],get_discovery(split_x,group,count_spaces) + discovered -1)] = "U"
                #new
                dict_of_texts[split_span[0]] -= 1
                dict_of_texts_2[split_span[0]] += 1
            else:
                index,dict_of_texts,dict_of_texts_2 = get_me_index(split_x,split_span[0],group,count_spaces,dict_of_texts,dict_of_texts_2,flag)
                y[index] = "U"
            #new change   
            discovered += 1
        

            
    targets.append(y)
    sentences.append(split_x)
    

In [401]:
loli = []
loli2 = []
l = 6971
for i in range(len(sentences[l])):
    new_first = re.findall(r"[\w]+|[\"\'*.,!?;]", sentences[l][i])
    
    if (len(new_first) == 3) and ("'" in new_first):
        if (new_first[1] == "'"):
            new_first = [new_first[0],new_first[1] + new_first[2]]
        
    loli.extend(new_first)
    
    if targets[l][i] == "I":
        loli2.extend(["I"] * len(new_first))
    elif (targets[l][i] == "L") :
        to_append = ["O"] * len(new_first)
        to_append[0] = "L"
        loli2.extend(to_append)
    elif (targets[l][i] == "B"):
        to_append = ["I"] * len(new_first)
        if (len(new_first) > 1) and ((new_first[0] == "'") or (new_first[0] == '"')):
            to_append[0] = "O"
            to_append[1] = "B"
        else:
            to_append[0] = "B"
        loli2.extend(to_append)
    elif (targets[l][i] == "U"):
        if len(new_first) == 1:
            loli2.extend("U")
        else:
            if "-" in new_first :
                to_append = ["I"] * len(new_first)
                to_append[0] = "B"
                to_append[-1] = "L"
                loli2.extend(to_append)
            elif new_first.count("'") == 2 and (new_first[1] != "'"):
                to_append = ["O"] * len(new_first)
                to_append[1] = "U"
                loli2.extend(to_append)
            else:
                to_append = ["O"] * len(new_first)
                to_append[0] = "U"
                loli2.extend(to_append)
    else:
        loli2.extend(["O"] * len(new_first))


In [ ]:
#for i,j in zip(loli,loli2):
#    print(i + "\t" + j)

In [13]:
answer_final_sentence = []
answer_final_span = []

for l in range(0,len(sentences)):
    if l == 4499:
        continue
    loli = []
    loli2 = []
    
    for i in range(len(sentences[l])):
        new_first = re.findall(r"[\w]+|[\"\'*.,!?;]", sentences[l][i])

        if (len(new_first) == 3) and ("'" in new_first):
            if (new_first[1] == "'"):
                new_first = [new_first[0],new_first[1] + new_first[2]]

        loli.extend(new_first)

        if targets[l][i] == "I":
            loli2.extend(["I"] * len(new_first))
        elif (targets[l][i] == "L") :
            to_append = ["O"] * len(new_first)
            to_append[0] = "L"
            loli2.extend(to_append)
        elif (targets[l][i] == "B"):
            to_append = ["I"] * len(new_first)
            if (len(new_first) > 1) and ((new_first[0] == "'") or (new_first[0] == '"')):
                to_append[0] = "O"
                to_append[1] = "B"
            else:
                to_append[0] = "B"
            loli2.extend(to_append)
        elif (targets[l][i] == "U"):
            if len(new_first) == 1:
                loli2.extend("U")
            else:
                if "-" in new_first :
                    to_append = ["I"] * len(new_first)
                    to_append[0] = "B"
                    to_append[-1] = "L"
                    loli2.extend(to_append)
                elif (new_first.count("'") == 2) and (new_first[1] != "'"):
                    to_append = ["O"] * len(new_first)
                    to_append[1] = "U"
                    loli2.extend(to_append)
                else:
                    to_append = ["O"] * len(new_first)
                    to_append[0] = "U"
                    loli2.extend(to_append)
        else:
            loli2.extend(["O"] * len(new_first))
            
    for f in range(len(loli2)):
        if (loli2[f] == "B") or (loli2[f] == "L") or (loli2[f] == "U"):
            loli2[f] = "I"
            
    answer_final_sentence.append(loli)
    answer_final_span.append(loli2)
    
    
with open("train_flair_IO.txt",'w',encoding = "utf-8") as f:
    
    for g in range(len(answer_final_sentence)):
        for word,tag in zip(answer_final_sentence[g],answer_final_span[g]):
            f.write(word+'\t'+tag+'\n')
                
        f.write('\n')